# Bibliotecas 

In [1]:
import sys
sys.path.append('/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/')


import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

import hepherolib.analysis as ana
from lgbm.controllers_lgb import LGBLearner, LGBModel

ana.start()
plt.style.use("default")

# Configuração

In [2]:
period = '18'
year_style = 2018
dataset_year = "2018"
basedir = '/eos/user/t/thenriqu/Dark_Matter/Amostras/hhdmAnalysis_deepJet_Regions/datasets'
dataset_name = basedir.split('/')[-2]

# Data folder
data_path = f"./data/{dataset_name}/{dataset_year}"

# Setup output folders
models_path = f"./models/{dataset_name}/{dataset_year}"
Path(models_path).mkdir(parents=True, exist_ok=True)

# Lendo os dados

In [3]:
model_name = "LGB_multi_signal"

X_train = pd.read_csv(f"{data_path}/{model_name}-train-data.csv")
X_test = pd.read_csv(f"{data_path}/{model_name}-test-data.csv")

W_train = X_train.pop("modelWeight").values
W_test = X_test.pop("modelWeight").values

Y_train = X_train.pop("Label").values
Y_test = X_test.pop("Label").values

evtWeight_train = X_train.pop("evtWeight").values
evtWeight_test = X_test.pop("evtWeight").values

In [4]:
X_train.head()

,LeadingLep_pt,LepLep_deltaM,LepLep_deltaR,LepLep_pt,MET_LepLep_Mt,MET_LepLep_deltaPhi,MET_pt,MT2LL,Nbjets,TrailingLep_pt
0,69.883340,0.582565,3.027906,40.11211,254.935670,2.920919,410.037570,26.969410,1,30.125252
1,114.942566,18.834435,1.757877,129.69420,122.296120,1.108255,104.121475,6.398953,2,33.967537
2,82.652370,5.285355,1.379278,118.73953,174.272480,2.980687,64.360050,85.835240,1,68.045020
3,245.787770,6.339325,0.763112,287.61877,274.435100,2.785732,67.581020,105.534900,1,51.919327
4,191.795600,3.473717,0.850083,240.17885,97.727554,0.878454,54.975773,22.928760,1,55.084420


In [5]:
W_train

array([1.41542031e-03, 1.03120503e-03, 3.54095349e-01, ...,
       1.72266674e-01, 5.73742776e+00, 2.18001609e-02])

In [6]:
Y_train

array([0, 0, 0, ..., 0, 1, 0])

# Train

In [7]:
path1 = '/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/Analysis_Regions/Analysis_2018/models/hhdmAnalysis_deepJet_Regions/2018/'


In [8]:
with open(path1 + 'LGB_multi_signal-bst-params.json', 'r') as f:
    best_params = json.load(f)

print(best_params)

{'subsample': 0.9, 'n_estimators': 9000, 'min_child_weight': 6, 'min_child_samples': 100, 'max_depth': 9, 'learning_rate': 0.008, 'eval_metric': 'error', 'colsample_bytree': 0.8}


In [9]:
params = { 'subsample': 0.9,
          'n_estimators': 9000,
          'min_child_weight': 6,
          'min_child_samples': 100,
          'max_depth': 9,
          'learning_rate': 0.008,
          'eval_metric': 'error',
          'colsample_bytree': 0.8
}

# Pode se ler através do arquivos .json salvo no código 2_find_hyperparams_lgb.ipynb
# params = json.load(open(f"{models_path}/{signal_name}-bst-params.json"))

num_boost_round = params.pop('n_estimators')
features = ["LeadingLep_pt", "LepLep_pt", "LepLep_deltaR", "LepLep_deltaM", "MET_pt", "MET_LepLep_Mt", "MET_LepLep_deltaPhi", "TrailingLep_pt", "MT2LL", "Nbjets"]

learner = LGBLearner(X_train, Y_train, W_train, features)
learner.train(params, num_boost_round)
learner.save_model(f"{models_path}/{model_name}-clf.model")
# learner.dump_model(f"{models_path}/{model_name}-raw.model")

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009961 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2302
[LightGBM] [Info] Number of data points in the train set: 2183562, number of used features: 10
[LightGBM] [Info] Start training from score 0.666667


# Nofiticar quando o Notebook terminar

In [10]:
from IPython.display import clear_output, display, HTML, Javascript

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Processo Completo!";
  window.speechSynthesis.speak(msg);
  alert("Processo Completo!")
"""))

<IPython.core.display.Javascript object>